In [74]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px  
from sklearn.preprocessing import LabelEncoder 
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,classification_report
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

In [75]:
#Reading the sample horse dataset and writing to a dataframe
df_horse=pd.read_csv(r"E:\DataScience\placement project\Race Pred\horses_1990.csv")
df_horse

,rid,horseName,age,saddle,decimalPrice,isFav,trainerName,jockeyName,position,positionL,...,TR,OR,father,mother,gfather,runners,margin,weight,res_win,res_place
0,271018,Combermere,6.0,0.0,0.222222,0,R G Frost,J Frost,1,NaN,...,94.0,NaN,Absalom,Queen's Parade,Sovereign Path,14,1.521003,69,1.0,1
1,271018,Royal Battery,6.0,0.0,0.090909,0,D H Barons,S Earle,2,10,...,88.0,NaN,Norfolk Air,All At Sea,Man The Rail,14,1.521003,69,0.0,1
2,271018,Just So,7.0,0.0,0.029412,0,J D Roberts,S Burrough,3,15,...,71.0,NaN,Sousa,Just Camilla,Ascertain I,14,1.521003,66,0.0,1
3,271018,Mandraki Shuffle,8.0,0.0,0.090909,0,Oliver Sherwood,M Richards,4,20,...,65.0,NaN,Mandalus,Indictment,Desert Call,14,1.521003,69,0.0,0
4,271018,Turnberry Dawn,8.0,0.0,0.047619,0,T B Hallett,P Richards,5,dist,...,45.0,NaN,Fair Turn,Shuil Alainn,Levanter,14,1.521003,69,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88190,211036,Whistle Blower I,5.0,0.0,0.076923,0,Henrietta Knight,G Upton,10,4,...,12.0,91.0,Le Bavard,Grangeclare Lady,Menelek,14,1.080962,64,0.0,0
88191,211036,Principal I,8.0,0.0,0.029412,0,P A Blockley,P Midgley,11,NaN,...,NaN,99.0,Deep Run,Lulu Dee,Straight Deal,14,1.080962,66,0.0,0
88192,211036,Nudge Double Up,5.0,0.0,0.038462,0,J A C Edwards,D Tegg,40,NaN,...,NaN,104.0,Hays,Bonny Bertha,Capistrano I,14,1.080962,71,0.0,0
88193,211036,Its All Over Now,6.0,0.0,0.029412,0,Anabel K Murphy,S Earle,40,NaN,...,NaN,86.0,Martinmas,Devon Lark,Take A Reef,14,1.080962,63,0.0,0


In [76]:
#Dropping the variables not meant to affect the final outcome
df_horse.drop(['RPR','TR','OR','gfather','father','mother','res_win','saddle', 'decimalPrice', 'isFav','weightSt','margin','weightLb','headGear'],axis=1, inplace=True)


In [67]:
df_horse.isna().sum()

rid                0
horseName          0
age               75
trainerName      145
jockeyName        52
position           0
positionL      34735
dist           25143
overWeight     83280
outHandicap    84663
runners            0
weight             0
res_place          0
dtype: int64

In [68]:
df_horse['positionL'].unique()

array([nan, '10', '15', '20', 'dist', '2.5', '.75', '12', '3', '5', '1.5',
       '2', 'shd', '4', '.5', 'hd', '1', '7', '6', '30', '8', 'nk', '0.5',
       '0.75', '25', '12.0', '6.0', '15.0', '4.0', '1.0', '2.0', '8.0',
       '3.0', '5.0', '10.0', '20.0', 'dht', '3.5', '1.75', '9', '11'],
      dtype=object)

In [69]:
import numpy as np
values_to_replace=['dist','shd','hd','nk','nan','dht']
df_horse['positionL'].replace(values_to_replace,np.nan,inplace=True)

C:\Users\Dell\AppData\Local\Temp\ipykernel_4300\3044959433.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_horse['positionL'].replace(values_to_replace,np.nan,inplace=True)


In [70]:
#Handling null value
df_horse=df_horse.dropna(subset=['age','dist','jockeyName','trainerName'])

In [71]:
df_horse['overWeight']=df_horse['overWeight'].fillna(0.0)
df_horse['outHandicap']=df_horse['outHandicap'].fillna(0.0)
df_horse['positionL']=df_horse['positionL'].fillna(0.0)

C:\Users\Dell\AppData\Local\Temp\ipykernel_4300\2269487395.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_horse['overWeight']=df_horse['overWeight'].fillna(0.0)
C:\Users\Dell\AppData\Local\Temp\ipykernel_4300\2269487395.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_horse['outHandicap']=df_horse['outHandicap'].fillna(0.0)
C:\Users\Dell\AppData\Local\Temp\ipykernel_4300\2269487395.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [72]:
df_horse['overWeight']=df_horse['overWeight'].astype(int)
df_horse['outHandicap']=df_horse['outHandicap'].astype(int)
df_horse['age']=df_horse['age'].abs().astype(int)

C:\Users\Dell\AppData\Local\Temp\ipykernel_4300\2079149196.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_horse['overWeight']=df_horse['overWeight'].astype(int)
C:\Users\Dell\AppData\Local\Temp\ipykernel_4300\2079149196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_horse['outHandicap']=df_horse['outHandicap'].astype(int)
C:\Users\Dell\AppData\Local\Temp\ipykernel_4300\2079149196.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [73]:
df_horse.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62926 entries, 2 to 88191
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   rid          62926 non-null  int64 
 1   horseName    62926 non-null  object
 2   age          62926 non-null  int32 
 3   trainerName  62926 non-null  object
 4   jockeyName   62926 non-null  object
 5   position     62926 non-null  int64 
 6   positionL    62926 non-null  object
 7   dist         62926 non-null  object
 8   overWeight   62926 non-null  int32 
 9   outHandicap  62926 non-null  int32 
 10  runners      62926 non-null  int64 
 11  weight       62926 non-null  int64 
 12  res_place    62926 non-null  int64 
dtypes: int32(3), int64(5), object(5)
memory usage: 6.0+ MB


In [12]:
df_horse['age']=df_horse['age'].abs().astype(int)

C:\Users\Dell\AppData\Local\Temp\ipykernel_4300\1137487749.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_horse['age']=df_horse['age'].abs().astype(int)


In [13]:
#Converting the dataframe columns into string datatype
df_horse=df_horse.astype(str)

In [52]:
#Reading the Races sample dataset and writing to dataframe
df_race=pd.read_csv(r"E:\DataScience\placement project\Race Pred\races_1990.csv")

In [53]:
#Dropping independant variables not affectinng the prediction
df_race.drop(['rclass','band','prizes','ncond'],axis=1, inplace=True)


In [54]:
df_race.isna().sum()

rid               0
course            0
time              0
date              0
title             0
ages           3553
distance          0
condition         0
hurdles        6474
winningTime       0
prize          1750
metric            0
countryCode       0
class             0
dtype: int64

In [56]:
#Function to convert the 'hurdles' column value into inches
def convert_to_inches(value):
    if 'fences' in value:
        number=int(value.split()[0])
        return number*12
    elif 'hurdles' in value:
        number=int(value.split()[0])
        return number*10
    else:
        return value

In [59]:
df_race['hurdles']=df_race['hurdles'].astype(str)

In [60]:
df_race['hurdles']=df_race['hurdles'].apply(convert_to_inches)

In [61]:
df_race['hurdles']=df_race['hurdles'].astype(float)

In [62]:
df_race['hurdles']=df_race['hurdles'].fillna(df_race['hurdles'].mean())

In [63]:
df_race['prize']=df_race['prize'].astype(float)


In [20]:
df_race['prize']=df_race['prize'].fillna(df_race['prize'].mean())

In [21]:
#Null handling
cols=['ages']
df_race.loc[:,cols] = df_race.loc[:,cols].ffill()

In [22]:
df_race['ages'] = df_race['ages'].str.extract(r'^(-?\d+)', expand=True).astype(int)

In [23]:
df_race['distance'] = df_race['distance'].str.extract(r'^(-?\d+)', expand=True).astype(int)


In [24]:
#Converting the dataframe into string
df_race=df_race.astype(str)

In [25]:
#Merging the Horses and Race dataframe into one based on 'rid' column
df_cml=pd.merge(df_horse, df_race, on='rid', how='inner')

In [26]:
df_cml['hurdles']=df_cml['hurdles'].astype(str)

In [27]:
#Creating  a smaller dataframe for model selection and training
df_new=df_cml.head(1000)

In [28]:
df_new.columns

Index(['rid', 'horseName', 'age', 'trainerName', 'jockeyName', 'position',
       'positionL', 'dist', 'overWeight', 'outHandicap', 'runners', 'weight',
       'res_place', 'course', 'time', 'date', 'title', 'ages', 'distance',
       'condition', 'hurdles', 'winningTime', 'prize', 'metric', 'countryCode',
       'class'],
      dtype='object')

In [29]:
#Label Encoding
cols=[ 'horseName', 'age', 'trainerName', 'jockeyName', 'position',
       'positionL', 'dist', 'overWeight', 'outHandicap', 'runners', 'weight',
       'res_place', 'course', 'time', 'date', 'title', 'ages', 'distance',
       'condition', 'hurdles', 'winningTime', 'prize', 'metric', 'countryCode',
       'class']
model=LabelEncoder()
for col in cols:
    df_new[col]=model.fit_transform(df_new[col])
df_new

C:\Users\Dell\AppData\Local\Temp\ipykernel_4300\1082960949.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[col]=model.fit_transform(df_new[col])
C:\Users\Dell\AppData\Local\Temp\ipykernel_4300\1082960949.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[col]=model.fit_transform(df_new[col])
C:\Users\Dell\AppData\Local\Temp\ipykernel_4300\1082960949.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

,rid,horseName,age,trainerName,jockeyName,position,positionL,dist,overWeight,outHandicap,...,title,ages,distance,condition,hurdles,winningTime,prize,metric,countryCode,class
0,271018,444,9,146,322,12,9,78,0,0,...,44,3,2,4,9,125,89,23,0,0
1,271018,521,10,327,186,13,12,157,0,0,...,44,3,2,4,9,125,89,23,0,0
2,271018,923,10,434,293,15,2,216,0,0,...,44,3,2,4,9,125,89,23,0,0
3,271018,979,10,138,373,16,11,217,0,0,...,44,3,2,4,9,125,89,23,0,0
4,275156,945,9,200,170,12,9,32,0,0,...,146,3,1,4,3,71,109,9,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,252682,705,6,410,41,12,5,196,0,0,...,72,1,1,0,10,28,26,9,0,0
996,252682,312,6,315,64,13,19,20,0,0,...,72,1,1,0,10,28,26,9,0,0
997,252682,719,6,278,104,15,14,33,0,0,...,72,1,1,0,10,28,26,9,0,0
998,252682,528,6,286,92,16,16,55,0,0,...,72,1,1,0,10,28,26,9,0,0


In [412]:
#classification model to find if the horse will win or not


In [30]:
#Model Building
X=df_new.drop(['res_place'],axis=1)
Y=df_new['res_place']
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

models=[LogisticRegression(),KNeighborsClassifier(),SVC(),DecisionTreeClassifier()]
for model in models:
    model.fit(x_train,y_train)
    train_pred=model.predict(x_train)
    test_pred=model.predict(x_test)
    print(f"-------------{type(model)}___name___-----------")
    print(f"acc: {accuracy_score(y_train,train_pred)}")
    print(f"pred: {precision_score(y_train,train_pred,average='micro')}")
    print(f"rec: {recall_score(y_train,train_pred,average='micro')}")
    print(f"f1: {f1_score(y_train,train_pred,average='micro')}")
    
    
    print(f"acctest: {accuracy_score(y_test,test_pred)}")
    print(f"predtest: {precision_score(y_test,test_pred,average='micro')}")
    print(f"rectest: {recall_score(y_test,test_pred,average='micro')}")
    print(f"f1test: {f1_score(y_test,test_pred,average='micro')}")

C:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


-------------<class 'sklearn.linear_model._logistic.LogisticRegression'>___name___-----------
acc: 0.89625
pred: 0.89625
rec: 0.89625
f1: 0.89625
acctest: 0.84
predtest: 0.84
rectest: 0.84
f1test: 0.84
-------------<class 'sklearn.neighbors._classification.KNeighborsClassifier'>___name___-----------
acc: 0.89625
pred: 0.89625
rec: 0.89625
f1: 0.89625
acctest: 0.84
predtest: 0.84
rectest: 0.84
f1test: 0.84
-------------<class 'sklearn.svm._classes.SVC'>___name___-----------
acc: 0.89625
pred: 0.89625
rec: 0.89625
f1: 0.89625
acctest: 0.84
predtest: 0.84
rectest: 0.84
f1test: 0.84
-------------<class 'sklearn.tree._classes.DecisionTreeClassifier'>___name___-----------
acc: 1.0
pred: 1.0
rec: 1.0
f1: 1.0
acctest: 0.995
predtest: 0.995
rectest: 0.995
f1test: 0.995
